In [11]:
#driver = webdriver.Chrome()

import numpy as np
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains


driver = webdriver.Chrome(ChromeDriverManager().install())

df = pd.DataFrame(columns=['carrier','Flight_id','departure_airport','arrival_airport', 'date', 'take_off_time','landing_time','flight_time','price'])

date_string = '2024-05-20'
airport_iata_departure="ROM"
airport_arrival='Bari'


# Créer une nouvelle instance du navigateur
#driver = webdriver.Chrome()

# Aller à la page de Ryanair
driver.get("https://www.ryanair.com/fr/fr")

try:
    # Attendre que le bouton "Oui, j’accepte" soit chargé
    button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".cookie-popup-with-overlay__button-settings[data-ref='cookie.accept-all']"))
    )
    # Cliquer sur le bouton
    button.click()
except Exception as e:
    print(f"Une erreur s'est produite : {e}")
try:
    # Cliquer sur le bouton "Aller simple"
    one_way_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "ry-radio-button[data-ref='flight-search-trip-type__one-way-trip']"))
    )
    one_way_button.click()
except Exception as e:
    print(f"Une erreur s'est produite : {e}")
try:
    departure_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#input-button__departure"))
    )
    departure_input.click()
    # Cliquer sur l'élément "Italie"
    italy_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/ry-tooltip/div[2]/hp-app-controls-tooltips/fsw-controls-tooltips-container/fsw-controls-tooltips/fsw-origin-container/fsw-airports/fsw-countries/div[3]/div[8]/span"))
    )
    buttonIT=driver.find_element(By.XPATH, "/html/body/ry-tooltip/div[2]/hp-app-controls-tooltips/fsw-controls-tooltips-container/fsw-controls-tooltips/fsw-origin-container/fsw-airports/fsw-countries/div[3]/div[8]/span")
    buttonIT.click()
    print("lllll")
    # Cliquer sur l'élément "Rome (Tous les aéroports)"
    wait = WebDriverWait(driver, 10)
    #rome_element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "span[data-ref='airport-item__mac-name'][data-id='ROM']")))
    rome_element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, f"span[data-ref='airport-item__mac-name'][data-id='{airport_iata_departure}']")))
    
    # Cliquer sur l'élément "Rome (Tous les aéroports)"
    driver.execute_script("arguments[0].click();", rome_element)
    print("zzzzzzzz")
    destination_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#input-button__destination"))
    )
    destination_input.clear()
    destination_input.send_keys(airport_arrival)
except Exception as e:
    print(f"Une erreur s'est produite : {e}")
try:
    # Attendre que le bouton "Recherche" soit visible
    search_button = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "button[data-ref='flight-search-widget__cta']"))
    )
    search_button.click()
except Exception as e:
    print(f"Une erreur s'est produite : {e}")

try:
    # Attendre que le bouton "Départ" soit visible
    departure_date = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "fsw-input-button[uniqueid='dates-from']"))
    )
    # Cliquer sur le bouton "Départ"
    departure_date.click()
    # Attendre que le bouton "mai" soit visible
    may_button = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "div.m-toggle__month[data-id='mai']"))
    )
    may_button.click()
    day_button = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, f"div.calendar-body__cell[data-id='{date_string}']"))
    )
    
    # Faire défiler l'élément en vue
    ActionChains(driver).move_to_element(day_button).perform()

    # Cliquez sur le bouton
    day_button.click()
    done_button = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "button.passengers__confirm-button[aria-label='Fini']"))
    )
    done_button.click()
except Exception as e:
    print(f"Une erreur s'est produite : {e}")
try:

    # Attendre que le bouton "Recherche" soit visible
    search_button = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "flight-search-widget__start-search-cta.ng-tns-c983940023-3.ry-button--gradient-yellow"))
    )
    #A retirer pour lancer la recherche
    #search_button.click()
except Exception as e:
    print(Exception)
    print(f"Une erreur s'est produite : {e}")



lllll
zzzzzzzz
Une erreur s'est produite : Message: element click intercepted: Element <div _ngcontent-ryanair-homepage-c2171854240="" class="calendar-body__cell" tabindex="0" data-id="2024-05-20" data-value="20" data-type="day">...</div> is not clickable at point (184, 573). Other element would receive the click: <div _ngcontent-ryanair-homepage-c101403801="" class="toast__text body-l-lg body-l-sm ng-tns-c101403801-15 ng-star-inserted" style="">...</div>
  (Session info: chrome=120.0.6099.109)



In [7]:
try:
    # Attendre que les éléments spécifiés soient visibles
    flight_list = WebDriverWait(driver, 10).until(
        EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "div.flight-card__wrapper"))
    )

    # Parcourir la liste des éléments et récupérer l'ensemble des données de chaque élément

    date_string = '2024-05-20'

    for flight in flight_list:
        my_flight=[]
        data = flight.text.split('\n')
        my_flight.append(data[0].replace('Opéré par ', ''))
        my_flight.append(data[3])
        my_flight.append(data[2])
        my_flight.append(data[6])
        my_flight.append(date_string)
        my_flight.append(data[1])
        my_flight.append(data[5])
        my_flight.append(data[4].replace(' h ', ':').replace(' m',''))
        my_flight.append(data[8].replace(' €', ''))
        print(my_flight)
        df.loc[len(df.index)] = my_flight
        df.to_csv('Ryanair_Italy_Flight_Data.csv', index=False)
except:
    print('Vol non enrregistré')


['Malta Air', 'FR 7075', 'Rome-Fiumicino', 'Bari', '2024-05-20', '17:45', '18:50', '1:5', '27,49']
['Malta Air', 'FR 7077', 'Rome-Fiumicino', 'Bari', '2024-05-20', '20:55', '22:00', '1:5', '27,49']
